# `fastapi-h5` publisher

This notebook will create FastAPI application publishing HSDS-like data via `fastapi-h5.router`.

The notebook uses `typing`, `contextlib` and `asyncio` from standard Python.<br>
Concerning the external dependencies, beside `fastapi` and `fastapi-h5`, it requires `uvicorn` and `numpy` (optional).

*This example demonstrates a basic communcation between 2 notebooks or Python processes without files and using [Hierarchical Data Format](https://www.hdfgroup.org/) (HDF) model.*

In [1]:
import time
from contextlib import nullcontext
from typing import Any, ContextManager, Tuple

import numpy as np
import uvicorn
from fastapi import FastAPI

from fastapi_h5 import router

- create FastAPI application
- add the `fastapi_h5.router`
- define a function or method returning data
- link that function to `app.state.get_data`
- configure uvicor server
- note that attributes are supported so [NeXus](https://www.nexusformat.org/) convention can be used
- run the server forewer (you can use the Notebook *interrupt* to stop the server)

In [2]:
app = FastAPI()

# add h5dict RestAPI port
app.include_router(router, prefix="/results")


# data published over HSDS-like interface
def get_data() -> Tuple[dict[str, Any], ContextManager[None]]:
    t = time.time()
    gmt = time.gmtime(t)
    data = {
        "entry": {
            "time": {
                "tm_year": gmt.tm_year,
                "tm_mon": gmt.tm_mon,
                "tm_mday": gmt.tm_mday,
                "tm_hour": gmt.tm_hour,
                "tm_min": gmt.tm_min,
                "tm_sec": gmt.tm_sec,
                "tm_zone": gmt.tm_zone,
            },
            "time_attrs": {"NX_class": "NXcollection"},
            "data": {
                "values": np.arange(10, dtype=np.float16),
            },
            "data_attrs": {"NX_class": "NXdata", "signal": "values"},
        },
        "entry_attrs": {"NX_class": "NXentry", "default": "data"},
        "_attrs": {"default": "entry"},
    }
    return data, nullcontext()


app.state.get_data = get_data

# configure uvicorn server
config = uvicorn.Config(app, host="0.0.0.0", port=8000, log_level="info")
server = uvicorn.Server(config)

# run forewer
await server.serve()

INFO:     Started server process [52925]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:55472 - "GET /results/?getdnids=1&getobjs=T&include_attrs=T&domain=%2F HTTP/1.1" 200 OK
INFO:     127.0.0.1:55472 - "GET /results/groups/g-h5dict-2F?domain=%2F HTTP/1.1" 200 OK
INFO:     127.0.0.1:55472 - "GET /results/groups/g-h5dict-2F/links/entry?CreateOrder=0&domain=%2F HTTP/1.1" 200 OK
INFO:     127.0.0.1:55472 - "GET /results/groups/g-h5dict-2F656E747279/links/time?CreateOrder=0&domain=%2F HTTP/1.1" 200 OK
INFO:     127.0.0.1:55472 - "GET /results/groups/g-h5dict-2F656E7472792F74696D65?domain=%2F HTTP/1.1" 200 OK
INFO:     127.0.0.1:55472 - "GET /results/groups/g-h5dict-2F656E7472792F74696D65/links/tm_hour?CreateOrder=0&domain=%2F HTTP/1.1" 200 OK
INFO:     127.0.0.1:55472 - "GET /results/datasets/d-h5dict-2F656E7472792F74696D652F746D5F686F7572?domain=%2F HTTP/1.1" 200 OK
INFO:     127.0.0.1:55472 - "GET /results/datasets/d-h5dict-2F656E7472792F74696D652F746D5F686F7572/value?domain=%2F HTTP/1.1" 200 OK
INFO:     127.0.0.1:55472 - "GET /results/datasets/d-h5dic

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [52925]
